### json 불러와서 저장하기

In [2]:
import numpy as np
import pandas as pd
import json
import jpype
from bs4 import BeautifulSoup as bs

data_path = "data"

json_train=[]
for i in range(0,18):
    json_file = json.loads(open("%s/%d.json" % (data_path, i)).read())
    json_train += json_file["articles"]

### json에서 author, author 형태소, forumid 추출

In [3]:
from konlpy.tag import Mecab

mecab = Mecab()

labeled_train = []

for cnt, article in enumerate(json_train):
    text = bs(article["text"],"lxml").text
    
    if cnt % 10000 == 0:
        print cnt

    labeled_train.append({
            "istroll": article["is_troll"],
            "author": article["author"],
            "author_morphs": mecab.morphs(article["author"]),
            "forumid": article["forumid"],
        })

labeled_train = pd.DataFrame.from_dict(labeled_train)
labeled_train

/Library/Python/2.7/site-packages/bs4/__init__.py:198: UserWarning: ".." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  '"%s" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.' % markup)
/Library/Python/2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://blog.naver.com/kakashi08/110086751501" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/Library/Python/2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://ilwar.com/free/5920
http://ilwar.com/free/6078" looks like a URL. Beautiful Soup is not an HTTP client. You should probabl

0
10000

/Library/Python/2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://image.kilho.net/?pk=1443527



Free" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/Library/Python/2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://www.youtube.com/watch?v=ByhP478ZPQI

ㄹㅇ존좋" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/Library/Python/2.7/site-packages/bs4/


20000

/Library/Python/2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://imgur.com/rBtm1LI.jpg" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/Library/Python/2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://m.ilwar.com/poli/17062" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/Library/Python/2.7/site-packages/bs4/__init__.py:207: UserWarning: "ht

...

/Library/Python/2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://m.khan.co.kr/view.html?category=2&med_id=khan&artid=201401071905352&code=361102" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/Library/Python/2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://ilwar.com/free/44658" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/Library/Python


50000

/Library/Python/2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://1.hidemyass.com/ip-9/encoded/Oi8vd3d3LnJvZG9uZy5yZXAua3Ava28v" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/Library/Python/2.7/site-packages/bs4/__init__.py:207: UserWarning: "http://www.ohmynews.com/NWS_Web/View/at_pg.aspx?CNTN_CD=A0001947743" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % m


60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000


,author,author_morphs,forumid,istroll
0,세부이장,"[세부, 이장]",마을회관밭,False
1,관리자,[관리자],잡담밭,False
2,snoopy,[snoopy],잡담밭,False
3,ISCARIOT,[ISCARIOT],정치사회밭,False
4,1234,[1234],연예인밭,False
5,호놀룰루,[호놀룰루],정치사회밭,False
6,영양죽,"[영, 양죽]",잡담밭,False
7,영양죽,"[영, 양죽]",연예인밭,False
8,fakebook,[fakebook],잡담밭,False
9,그나,[그나],잡담밭,False


### forumid와 author을 label encoder로 숫자로 바꿔줌

In [4]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

labeled_train["forumid"] = le.fit_transform(labeled_train["forumid"])
labeled_train["author"] = le.fit_transform(labeled_train["author"])

labeled_train

,author,author_morphs,forumid,istroll
0,15848,"[세부, 이장]",16,False
1,5653,[관리자],31,False
2,3752,[snoopy],31,False
3,1258,[ISCARIOT],32,False
4,136,[1234],23,False
5,26299,[호놀룰루],32,False
6,18849,"[영, 양죽]",31,False
7,18849,"[영, 양죽]",23,False
8,2693,[fakebook],31,False
9,6140,[그나],31,False


###bag of words

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

max_features = 1000

vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None,
                            stop_words = None, max_features=max_features)

train_data_features = vectorizer.fit_transform((labeled_train["author_morphs"].apply(lambda x:" ".join(x)))).toarray()

col = ["bow_author_%s" % data for data in vectorizer.get_feature_names()]
labeled_train = pd.concat([labeled_train, pd.DataFrame(train_data_features, columns = col, index=labeled_train.index)],axis=1)

### predictor, model 생성

In [6]:
from sklearn.ensemble import RandomForestClassifier

label = 'istroll'
pre = labeled_train.columns.drop(['author_morphs', 'author', label])
#pre = ['author','forumid']

model = RandomForestClassifier(n_estimators=10, n_jobs=6)

###cross validation

In [7]:
from sklearn import cross_validation

cv_value = 3

scores = cross_validation.cross_val_score(model, labeled_train[pre], labeled_train[label], cv=cv_value, scoring="roc_auc")
cv_result = scores.mean()

print(cv_result)

0.736101517807


## Result

 * author를 Label Encoder로 변환: 0.67331271058
 * author를 Bag of Words(feature = 1,000 개)로 변환: 0.733333689652

In [10]:
import pickle

pre = labeled_train.columns.drop(['author_morphs', 'author'])

pickle.dump(labeled_train[pre], open("author, forum_%d.p" % max_features, "wb"), protocol = pickle.HIGHEST_PROTOCOL)